# Convert model data for webapp

This notebook prepares data and saves it in Javascript/JSON format for use by the webapp.

1. The numerical model is translated into Javascript code, and saved as `model.js`.
2. The Sankey diagrams are drawn and saved to a JSON file.
3. The lever workbook is read and the processed contents saved to a JSON file.

In [ ]:
import numpy as np
import logging
import re
import pandas as pd
import json
import sympy as sy

In [ ]:
%load_ext autoreload
%autoreload 2

## Define levers

Currently just define them directly here in the notebook and write to JSON. Could parse these from elsewhere in future to make it easier to manage.

In [ ]:
from load_levers import read_levers
levers = read_levers("levers.xlsx")

In [ ]:
# TODO the translation from X[0] to X__0 is Javascript model specific and should be done here

In [ ]:
#levers

In [ ]:
#defaults

In [ ]:
with open(f"/Users/rcl38/work/cthru/global-petrochemicals-calculator-app/static/levers.json", "wt") as f:
    json.dump(levers.to_dict(), f, indent=2)

## Model

In [ ]:
import load_model
model_data = load_model.load_model()

In [ ]:
model, recipe_data = load_model.build_model(model_data)
other_results = load_model.define_model(model, recipe_data, levers)

In [ ]:
flows_sym = model.to_flows(recipe_data, flow_ids=True)
func = model.lambdify(recipe_data)

In [ ]:
from sympy import Indexed
from sympy.printing.jscode import jscode

#def indexed_to_single_symbol(sym):
#    return re.sub(r"^(.+)[[](.+)[]]$", r"~~\1~~\2~~", str(sym))

#def single_symbols_to_indexed(s):
#    new, _ = re.subn(r"~~([^~]+)~~([^~]+)~~", r"\1[\2]", s)
#    return new

def indexed_to_single_symbol(sym):
    return re.sub(r"^(.+)[[](.+)[]]$", r"\1__\2", str(sym))

def single_symbols_to_indexed(s):
    return s

def to_js_module(values, intermediates, data_for_intermediates):
    symbols = set()
    items = []
    for id, value in values.items():
        value = value.xreplace(recipe_data)
        value = value.xreplace({
            x: sy.Symbol(indexed_to_single_symbol(x)) for x in value.free_symbols if isinstance(x, Indexed)
        })
        symbols.update({str(x) for x in value.free_symbols if not isinstance(x, sy.Indexed)})
        items.append(
            f'"{id}": ({single_symbols_to_indexed(jscode(value))}),'
        )

    # Substitute recipe in intermediates now
    # FIXME double substitution
    intermediates = [
        (sym, expr.xreplace(data_for_intermediates).xreplace(data_for_intermediates))
        for sym, expr, _ in intermediates
    ]
    
    intermediates = [
        (sym, expr.xreplace({
            x: sy.Symbol(indexed_to_single_symbol(x)) for x in expr.free_symbols if isinstance(x, Indexed)
        }))
        for sym, expr in intermediates
    ]
    
    for sym, expr in intermediates:
        symbols.update({str(x) for x in expr.free_symbols if not isinstance(x, sy.Indexed)})

    for sym, expr in intermediates:
        symbols -= {str(sym)}
        
    intermediates_str = single_symbols_to_indexed("\n".join([
        "const " + str(sym) + " = " + jscode(expr) + ";"
        for sym, expr in intermediates
    ]))
    args_list = ", ".join(symbols)
    body = "\n        ".join(items)
    s = """
// Generated from Python model
export default function(params) {
    const Max = Math.max;
    const {%s} = params;
    %s
    return {
        %s
    }
}
""" % (args_list, intermediates_str, body)
    return s

In [ ]:
flows_sym_values = {row.id: row.value for _, row in flows_sym.iterrows()}

In [ ]:
s = to_js_module({**flows_sym_values, **other_results}, model._intermediates, recipe_data)
#print(s)

In [ ]:
with open("/Users/rcl38/work/cthru/global-petrochemicals-calculator-app/lib/model.js", "wt") as f:
    f.write(s)

## Sankey diagrams

In [ ]:
baseline = {lever.lever_id: lever.levels[0].level_id for lever in levers.levers}
data_for_sankey = levers.get_params(baseline, 0)

In [ ]:
from sankey_definitions import sdd_overall, sdd_lifecycle, sdd_eol, sdd_chemical_synthesis, sdd_primary, sdd_feedstock, palette
from floweaver import weave, Dataset

In [ ]:
# This is necessary so that the list of flow_ids is saved into JSON
#
# FIXME wouldn't need to customise this here if floweaver saved original_flows into JSON

def measures(group):
    # if "4447a79b718289f47e017f3843a6a7aa" in group.index:
    #     print(group.iloc[0])
    return {
        "value": group["value"].sum(),
        "flow_ids": list(group.index),
    }

def link_width(data):
    return data["value"]

In [ ]:
# Work around dodgy floweaver behaviour -- flows index is reset when Dataset
# is created. This only works because not using dim_process etc.
dataset = Dataset(flows_sym)
dataset._flows = dataset._table = flows_sym.set_index("id")

sdds = {
    "overall": sdd_overall,
    "synthesis": sdd_chemical_synthesis,
    "lifecycle": sdd_lifecycle,
    "primary": sdd_primary,
    "feedstock": sdd_feedstock,
    "eol": sdd_eol,
}

baseline_lever_settings = {lever.lever_id: lever.levels[0].level_id for lever in levers.levers}
test_params = levers.get_params(baseline_lever_settings, time_index=0)


sankey_data = {
    k: weave(sdd, dataset, measures=measures, link_width=link_width, palette=palette)
    for k, sdd in sdds.items()
}

sankey_data_with_data = {
    k: load_model.subs_in_sankey_data(d, func, test_params)
    for k, d in sankey_data.items()
}

In [ ]:
!mkdir -p sankeys

In [ ]:
sankey_data_with_data["overall"].to_widget(width=1000, height=600).auto_save_png("sankeys/overall.png")

In [ ]:
sankey_data_with_data["synthesis"].to_widget(width=1000, height=600).auto_save_png("sankeys/synthesis.png")

In [ ]:
sankey_data_with_data["lifecycle"].to_widget(width=1000, height=400).auto_save_png("sankeys/lifecycle.png")

In [ ]:
sankey_data_with_data["primary"].to_widget(width=1000, height=600).auto_save_png("sankeys/primary.png")

In [ ]:
sankey_data_with_data["feedstock"].to_widget(width=1000, height=400).auto_save_png("sankeys/feedstock.png")

In [ ]:
sankey_data_with_data["eol"].to_widget(width=1000, height=400).auto_save_png("sankeys/eol.png")

In [ ]:
import json

sankey_scales = {
    "overall": 1e-7,
    "synthesis": 2e-7,
    "lifecycle": 1e-7,
    "primary": 1e-7,
    "feedstock": 1e-7,
    "eol": 1e-7,
}

with open(f"/Users/rcl38/work/cthru/global-petrochemicals-calculator-app/static/diagrams_bundle.json", "wt") as f:
    json.dump({
        k: {
            "id": k,
            "shortRef": k,
            "label": k.title(),
            "ancestors": [],
            "parent": None,
            "children": [],
            # Strip "value" which is a Sympy expression which is not serialisable. The value will be added back by the model.
            "diagram": load_model.reset_value_in_sankey_data(sankey_data[k]).to_json(format="widget"),
            "scale": sankey_scales[k],
        }
        for k in sdds
    }, f, indent=2)